In [3]:
import json

with open('/home/infidea/workspace/krx_benchmark/data/krx_textbook/processed.json', 'r') as f:
    original_json = json.load(f)


In [4]:
original_json

[{'term': '금융',
  'description': '금융 (Finance) 이란 돈이 남는 사람과 돈이 모자라는 사람 간에 돈 (자금)을 돌려쓰는 것을 말합니다. 따라서 돈이 남는 사람은 자금의 공급자로서의 역할을, 돈이 모자라는 사람은 자금의 수요자로서의 역할을 하게 됩니다. 자금은 일반적으로 자금의 공급자인 가계부문으로부터 자금의 수요자인 기업이나 정부로 흘러갑니다. 이러한 자금거래가 이루어지기 위해서는 매개수단이 필요한데 이를 금융상품이라고 하며, 은행 등이 자금융통을 위해 사용하는 예금 및 적금, 주식 및 채권과 같은 증권, 그리고 선물 및 옵션과 같은 파생상품이 있습니다.'},
 {'term': '금융시장',
  'description': '금융시장은 금융상품의 거래가 조직적으로 이루어지는 시장으로서 가계, 기업, 정부, 금융기관 등 경제주체들이 필요한 자금을 조달하거나 여유자금을 운용하는 곳이라고 할 수 있습니다. 이러한 금융시장은 경제주체들이 저축, 차입, 보험계약 등을 통해 소비나 투자와 같은 경제활동을 원활히 하도록 하고, 생산성이 더 높은 경제활동의 영역으로 자금이 흘러가도록 함으로써 자원배분의 효율성을 높이기도 합니다. 또한 금융시장은 유휴자금을 동원하여 기업의 설립을 가능케 한다는 점에서 경제적 생산활동도 뒷받침하고 있습니다.'},
 {'term': '금융문맹',
  'description': '일상생활과 산업분야에서 금융이 차지하는 비중과 중요성이 커지면서 금융 관련 지식 부족으로, 돈을 제대로 관리하지 못하는 금융문맹이 될 경우 글자를 읽고 쓸 줄 모르는 문맹과 같이 국민 개개인의 삶의 질 뿐만 아니라 사회성장기반도 약화될 수 있음을 이야기할 때 쓰이는 용어이다. 금융문맹이라는 용어는 1990년대 미국에서 등장하였는데, 당시 미국 경제가 사상 최장기의 고성장을 지속하였음에도 한편으로 지속적인 저축률 저하, 민간부채 증가, 개인파산 급증 등 경제ㆍ사회 문제가 심화되었다. 이는 근본적으로 돈의 소중함과 관리방식을 모르는 ‘금융문맹

In [20]:
def generate_quiz_prompt(term, description, answer):
    prompt = f"""
당신은 금융 문제 생성 전문가입니다.

입력 데이터:
용어: {term}
설명: {description}
정답 위치: {answer}

다음 요구사항에 맞춰 객관식 문제를 생성해주세요:

1. 문제 요구사항:
   - '{term}'의 정의나 핵심 특성을 물어보는 문제여야 함
   - 명확하고 간단한 문장으로 작성
   - 부정문 사용 지양

2. 선택지 요구사항:
   - 총 4개의 선택지 필요
   - {answer}번이 정답이 되도록 구성
   - 오답은 설명의 내용을 살짝 변형하여 그럴듯하게 작성
   - 각 선택지는 비슷한 길이로 구성
   - 선택지 간 중복되는 내용 없도록 작성

출력 형식 (JSON):
{{
    "question": "문제 내용",
    "A": "첫 번째 선택지",
    "B": "두 번째 선택지",
    "C": "세 번째 선택지",
    "D": "네 번째 선택지",
    "answer": "{answer}"
}}

주의사항:
- 모든 선택지는 완전한 한국어 문장으로 작성
- 정답과 오답의 난이도 차이가 적절해야 함
"""
    return prompt


In [ ]:
GPT_API_KEY = "sk-XXX"
import openai
client = openai.OpenAI(api_key=GPT_API_KEY)

from pydantic import BaseModel
class QuestionModel(BaseModel):
    question: str
    A: str
    B: str
    C: str
    D: str
    answer: str


In [22]:
def make_question(data):
    import random
    res_list = []
    for datum in data:
        term = datum['term']
        description = datum['description']
        answer_key = random.choice(["A","B","C","D"])
        prompt = generate_quiz_prompt(term, description, answer_key)
        response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who is good at extracting information from text with json format."},
            {"role": "user", "content": prompt},
        ],
        response_format=QuestionModel,
        )
        answer  = response.choices[0].message.content
        print(answer, type(answer)) 
        answer_str = json.loads(answer)
        res_list.append(answer_str)
    

In [23]:
make_question(original_json)

{"question":"금융이란 무엇을 의미합니까?","A":"돈이 남는 사람과 모자라는 사람 간에 자금을 돌려쓰는 것","B":"은행과 같은 금융기관이 모든 돈을 소유하는 것","C":"가계부문에서만 자금을 차용하는 것","D":"선물과 옵션만을 사용하는 금융거래","answer":"A"} <class 'str'>
{"question":"금융시장의 역할은 무엇인가요?","A":"경제주체들이 자금을 조달하거나 운용할 수 있도록 돕습니다.","B":"주식과 채권만 거래하는 시장입니다.","C":"개인만을 위한 자금 운용 시장입니다.","D":"금융시장은 주로 저축을 제한하는 역할을 합니다.","answer":"A"} <class 'str'>
{"question":"‘금융문맹’이 국민 개개인의 삶과 사회에 미치는 영향을 설명하는 올바른 정의는 무엇인가요?","A":"금융문맹은 금융 지식을 바탕으로 자산을 잘 관리하는 능력입니다.","B":"금융문맹은 금융 지식 부족으로 개인의 삶의 질과 사회 성장 기반을 약화시킬 수 있습니다.","C":"금융문맹은 경제 성장과 함께 개인 저축률과 민간부채가 동시에 증가하는 현상입니다.","D":"금융문맹은 금융 교육을 통해 경제 문제를 모두 해결할 수 있다는 믿음입니다.","answer":"B"} <class 'str'>
{"question":"다음 중 간접금융의 특징으로 올바른 것은?","A":"자금의 수요자가 주식이나 채권을 직접 발행하여 자금을 조달한다.","B":"금융중개기관이 자신의 신용을 통해 자금을 융통해준다.","C":"자금의 수요자가 직접 자금을 공급하는 기관과 거래한다.","D":"기업과 정부가 직접 금융시장에서 필요한 자금을 조달한다.","answer":"B"} <class 'str'>
{"question":"대출시장이란 무엇을 의미하나요?","A":"대출시장은 정부가 자금을 빌려주는 시장을 말합니다.","B":"대출시장은 주식 시장과 연관된 자금 조달을 담당합니다.","C":"대출시장은 예금 금융기관을 통해 

In [32]:
with open('question.jsonl','r') as f:
    questions = f.readlines()

In [33]:
questions = [json.loads(q) for q in questions]

In [34]:
questions

[{'question': '금융이란 무엇을 의미합니까?',
  'A': '돈이 남는 사람과 모자라는 사람 간에 자금을 돌려쓰는 것',
  'B': '은행과 같은 금융기관이 모든 돈을 소유하는 것',
  'C': '가계부문에서만 자금을 차용하는 것',
  'D': '선물과 옵션만을 사용하는 금융거래',
  'answer': 'A'},
 {'question': '금융시장의 역할은 무엇인가요?',
  'A': '경제주체들이 자금을 조달하거나 운용할 수 있도록 돕습니다.',
  'B': '주식과 채권만 거래하는 시장입니다.',
  'C': '개인만을 위한 자금 운용 시장입니다.',
  'D': '금융시장은 주로 저축을 제한하는 역할을 합니다.',
  'answer': 'A'},
 {'question': '‘금융문맹’이 국민 개개인의 삶과 사회에 미치는 영향을 설명하는 올바른 정의는 무엇인가요?',
  'A': '금융문맹은 금융 지식을 바탕으로 자산을 잘 관리하는 능력입니다.',
  'B': '금융문맹은 금융 지식 부족으로 개인의 삶의 질과 사회 성장 기반을 약화시킬 수 있습니다.',
  'C': '금융문맹은 경제 성장과 함께 개인 저축률과 민간부채가 동시에 증가하는 현상입니다.',
  'D': '금융문맹은 금융 교육을 통해 경제 문제를 모두 해결할 수 있다는 믿음입니다.',
  'answer': 'B'},
 {'question': '다음 중 간접금융의 특징으로 올바른 것은?',
  'A': '자금의 수요자가 주식이나 채권을 직접 발행하여 자금을 조달한다.',
  'B': '금융중개기관이 자신의 신용을 통해 자금을 융통해준다.',
  'C': '자금의 수요자가 직접 자금을 공급하는 기관과 거래한다.',
  'D': '기업과 정부가 직접 금융시장에서 필요한 자금을 조달한다.',
  'answer': 'B'},
 {'question': '대출시장이란 무엇을 의미하나요?',
  'A': '대출시장은 정부가 자금을 빌려주는 시장을 말합니다.',
  'B': '대출시장은 주식 

In [39]:
res_list = []
for q,d in zip(questions,original_json):
    question = q['question']+'\nA:'+q['A']+'\nB:'+q['B']+'\nC:'+q['C']+'\nD:'+q['D']
    answer = q['answer']
    description = d['description']
    prompt = f"""
    주어진 문제, 설명, 정답을 바탕으로 생각의 사슬 형식으로 풀이를 생성해 주세요.
    문제: {question}
    설명: {description}
    정답: {answer}
    
    지시사항 : 
    1. 문제를 풀이하는 과정을 설명을 참고해 총 3문장 이하의 생각의 사슬 형식으로 작성해 주세요.
    2. '따라서 정답은 {answer}입니다' 로 마무리해 주세요.
    다른 설명은 없이 풀이만 작성해 주세요.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        )
    response_txt  = response.choices[0].message.content
    print(f"### LABEL_ANSER = {answer}")
    print(response_txt.split('따라서')[-1])
    answer_dict = {
        'question':question,
        'answer':response_txt
    }    
    res_list.append(answer_dict)

### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = A
 정답은 A입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = C
 정답은 C입니다.
### LABEL_ANSER = D
 정답은 D입니다.
### LABEL_ANSER = B
 정답은 B입니다.
### LABE

In [40]:
res_list

[{'question': '금융이란 무엇을 의미합니까?\nA:돈이 남는 사람과 모자라는 사람 간에 자금을 돌려쓰는 것\nB:은행과 같은 금융기관이 모든 돈을 소유하는 것\nC:가계부문에서만 자금을 차용하는 것\nD:선물과 옵션만을 사용하는 금융거래',
  'answer': '금융이란 자금이 남는 사람과 모자라는 사람 간에 자금을 순환시키는 것을 의미합니다. 설명에 따르면, 자금의 흐름은 공급자와 수요자 간의 상호작용으로 이루어지며, 이를 위한 다양한 금융상품이 존재합니다. 따라서 정답은 A입니다.'},
 {'question': '금융시장의 역할은 무엇인가요?\nA:경제주체들이 자금을 조달하거나 운용할 수 있도록 돕습니다.\nB:주식과 채권만 거래하는 시장입니다.\nC:개인만을 위한 자금 운용 시장입니다.\nD:금융시장은 주로 저축을 제한하는 역할을 합니다.',
  'answer': '금융시장은 가계, 기업, 정부, 금융기관 등 다양한 경제주체들이 자금을 조달하거나 운용할 수 있도록 돕는 곳입니다. 금융시장은 자원이 효율적으로 배분되도록 하여 경제활동의 생산성을 높이고, 기업의 설립을 촉진시킵니다. 따라서 정답은 A입니다.'},
 {'question': '‘금융문맹’이 국민 개개인의 삶과 사회에 미치는 영향을 설명하는 올바른 정의는 무엇인가요?\nA:금융문맹은 금융 지식을 바탕으로 자산을 잘 관리하는 능력입니다.\nB:금융문맹은 금융 지식 부족으로 개인의 삶의 질과 사회 성장 기반을 약화시킬 수 있습니다.\nC:금융문맹은 경제 성장과 함께 개인 저축률과 민간부채가 동시에 증가하는 현상입니다.\nD:금융문맹은 금융 교육을 통해 경제 문제를 모두 해결할 수 있다는 믿음입니다.',
  'answer': '금융문맹은 금융 지식 부족이 개인의 자산 관리에 악영향을 미쳐, 삶의 질 저하와 사회 성장 기반 약화를 초래할 수 있습니다. 이러한 현상은 글자 문맹과 유사한 영향을 미치며, 1990년대 미국에서 경제 문제가 심화되면서 주목받기 시작했습니다. 따라서 정답은 B입니다.

In [41]:
from datasets import Dataset

dataset = Dataset.from_list(res_list)

In [44]:
len(dataset)

212

In [43]:
dataset.push_to_hub('overfit-brothers/krx-textbook', private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/overfit-brothers/krx-textbook/commit/7130e8ba05eaf4bae46b02dfb07715f422d0ce79', commit_message='Upload dataset', commit_description='', oid='7130e8ba05eaf4bae46b02dfb07715f422d0ce79', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/overfit-brothers/krx-textbook', endpoint='https://huggingface.co', repo_type='dataset', repo_id='overfit-brothers/krx-textbook'), pr_revision=None, pr_num=None)